In [11]:
import pandas as pd
from pandasql import sqldf
import pyparsing as pyp
from os import path
import string

DATA_IN = '../../data/raw/Jefferson_County_KY_Street_Intersections.csv'
assert path.exists(DATA_IN)

df = pd.read_csv(DATA_IN)



In [12]:
assert df.OBJECTID.is_unique
assert df.INTID.is_unique

reindex = df.set_index("INTID")
reindex.head(3)

,X,Y,OBJECTID,SIFCODE1,SIFCODE2,SCCAD_ID,FST_INTPRE,FST_INTNAME,FST_INTSUF,SEC_INTPRE,SEC_INTNAME,SEC_INTSUF,X_COORD,Y_COORD,FST_SIFID,SEC_SIFID,GLOBALID
INTID,,,,,,,,,,,,,,,,,
154647662,-85.510444,38.205887,1,5464,7662,1,,REHL,RD,W,REHL,CT,1.278243e+06,259531.93750,4976,6856,{CD0D9D51-41FB-46FF-B229-AC9C0DDB7E61}
254646551,-85.528169,38.200376,2,5464,6551,2,,REHL,RD,,TUCKER STATION,RD,1.273126e+06,257588.25000,4976,5908,{A9FAED81-F7AE-436F-A657-F95FE1B905E8}
354646551,-85.528419,38.200360,3,5464,6551,3,,REHL,RD,,TUCKER STATION,RD,1.273051e+06,257590.85375,4976,5908,{71155AF3-3487-4EDA-8B72-B282378DE7F3}


In [13]:
keepers = list()
dropping = list()

def Keep(*names):
     keepers.extend(names)

def Drop(*names):
     dropping.extend(names)

Keep('X', 'Y')
Drop('X_COORD', 'Y_COORD',)

Drop('OBJECTID')

Drop('SIFCODE1', 'SIFCODE2', 
     'FST_SIFID', 'SEC_SIFID',)

('INTID')

Drop('SCCAD_ID')

Keep('FST_INTNAME','SEC_INTNAME')

Keep('FST_INTPRE', 'FST_INTSUF', 
     'SEC_INTPRE', 'SEC_INTSUF', )

Drop('GLOBALID')

keepers, dropping

(['X',
  'Y',
  'FST_INTNAME',
  'SEC_INTNAME',
  'FST_INTPRE',
  'FST_INTSUF',
  'SEC_INTPRE',
  'SEC_INTSUF'],
 ['X_COORD',
  'Y_COORD',
  'OBJECTID',
  'SIFCODE1',
  'SIFCODE2',
  'FST_SIFID',
  'SEC_SIFID',
  'SCCAD_ID',
  'GLOBALID'])

In [14]:
df.columns

Index(['X', 'Y', 'OBJECTID', 'SIFCODE1', 'SIFCODE2', 'INTID', 'SCCAD_ID',
       'FST_INTPRE', 'FST_INTNAME', 'FST_INTSUF', 'SEC_INTPRE', 'SEC_INTNAME',
       'SEC_INTSUF', 'X_COORD', 'Y_COORD', 'FST_SIFID', 'SEC_SIFID',
       'GLOBALID'],
      dtype='object')

In [15]:
dropped = df.drop(dropping, axis=1)

# Renames reorder
renames = {'X':'longitude',
           'Y':'latitude',
              'INTID':'intid',
              'FST_INTPRE':'main_prefix',
              'FST_INTNAME':'main_street', 
              'FST_INTSUF':'main_suffix',
              'SEC_INTPRE':'cross_prefix',
              'SEC_INTNAME':'cross_street',
              'SEC_INTSUF':'cross_suffix'}

renamed = dropped.rename(renames, axis=1)

rnkeepers = list(renames.values())
renamed

,longitude,latitude,intid,main_prefix,main_street,main_suffix,cross_prefix,cross_street,cross_suffix
0,-85.510444,38.205887,154647662,,REHL,RD,W,REHL,CT
1,-85.528169,38.200376,254646551,,REHL,RD,,TUCKER STATION,RD
2,-85.528419,38.200360,354646551,,REHL,RD,,TUCKER STATION,RD
3,-85.504954,38.222603,431949996,,I 64 EAST,,,I 265 RAMP,
4,-85.505546,38.222127,593499996,,I 265 NORTH,,,I 265 RAMP,
...,...,...,...,...,...,...,...,...,...
20823,-85.863923,38.185293,30157F677F675,,MAPLE GROVE,PL,,LAUREL STAND,WAY
20824,-85.864155,38.185290,30155F676F675,,WOODLAND MILLS,DR,,LAUREL STAND,WAY
20825,-85.864536,38.187825,30156F6763703,,WOODLAND MILLS,DR,,LADD,AVE
20826,-85.897758,38.079360,30158E806F678,,TRAIN STATION,WAY,,RAILCAR,RD


In [16]:
clean = renamed.set_index('intid')

In [17]:
clean[['main_street', 'cross_street', 'main_suffix', 'cross_suffix', 'cross_prefix', 'main_prefix', 'longitude', 'latitude']]

,main_street,cross_street,main_suffix,cross_suffix,cross_prefix,main_prefix,longitude,latitude
intid,,,,,,,,
154647662,REHL,REHL,RD,CT,W,,-85.510444,38.205887
254646551,REHL,TUCKER STATION,RD,RD,,,-85.528169,38.200376
354646551,REHL,TUCKER STATION,RD,RD,,,-85.528419,38.200360
431949996,I 64 EAST,I 265 RAMP,,,,,-85.504954,38.222603
593499996,I 265 NORTH,I 265 RAMP,,,,,-85.505546,38.222127
...,...,...,...,...,...,...,...,...
30157F677F675,MAPLE GROVE,LAUREL STAND,PL,WAY,,,-85.863923,38.185293
30155F676F675,WOODLAND MILLS,LAUREL STAND,DR,WAY,,,-85.864155,38.185290
30156F6763703,WOODLAND MILLS,LADD,DR,AVE,,,-85.864536,38.187825


# Run All to write csv. 

Next cell does the last step

In [19]:
cleaned = clean[['main_street', 'cross_street', 'main_suffix', 'cross_suffix', 'cross_prefix', 'main_prefix', 'longitude', 'latitude']]
DATA_OUT  = '../../data/clean/Jefferson_County_KY_Street_Intersections.csv'
cleaned.to_csv(DATA_OUT)